In [1]:
import cv2, os, time, keras
import numpy as np
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

Using TensorFlow backend.


In [2]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.50)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

In [3]:
model = models.load_model("resnet50_csv_15Face.h5", backbone_name='resnet50')
model = models.convert_model(model)
print(model.summary())
labels_to_names = {0: 'Face'}

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_1:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_2:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_3:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_4:0' shape=(9, 4) dtype=float32> anchors
Model: "retinanet-bbox"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 

In [4]:
cap = cv2.VideoCapture("istockphoto-667568266-640_adpp_is.mp4")
#cap.set(cv2.CAP_PROP_FRAME_WIDTH, 800)
#cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 600)
while True:
    try:
        ret, image = cap.read()
        draw = image.copy()
        #draw = cv2.resize(draw,(640,480))
        draw = cv2.resize(draw,(640,360))
        draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)
        image = preprocess_image(image)
        image, scale = resize_image(image)
        boxes, scores, labels = model.predict(np.expand_dims(image, axis=0))

        boxes /= scale

        for box, score, label in zip(boxes[0], scores[0], labels[0]):
            if score < 0.5:
                break
            color = label_color(label)
            b = box.astype(int)
            x = b[0]
            y = b[1]
            draw_box(draw, b, color=color)
            caption = "{} {:.3f}".format(labels_to_names[label], score)
            draw_caption(draw, b, caption)

        draw = cv2.cvtColor(draw, cv2.COLOR_RGB2BGR)
        cv2.imshow('framename', draw)
    except:
	    pass

    if cv2.waitKey(25) & 0xFF == ord('q'):
      break
    
sess.close()
cap.release()
cv2.destroyAllWindows()

